# About
Convert MobileBERT_SQuAD to Core ML

Try to optimize the TF graph then save as SavedModel.
Failed with error:
```
ValueError: Input 0 of node Cast was passed float from input_mask:0 incompatible with expected int32.
```


# Download model and setup enviroments

In [1]:
!wget https://storage.googleapis.com/cloud-tpu-checkpoints/mobilebert/mobilebert_squad_savedmodels.tar.gz
!tar -zxvf mobilebert_squad_savedmodels.tar.gz

--2021-08-18 13:23:32--  https://storage.googleapis.com/cloud-tpu-checkpoints/mobilebert/mobilebert_squad_savedmodels.tar.gz
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.0.48, 172.217.164.176, 172.217.2.112, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.0.48|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 184730530 (176M) [application/octet-stream]
Saving to: ‘mobilebert_squad_savedmodels.tar.gz’

mobilebert_squad_sa 100%[===================>] 176.17M   104MB/s    in 1.7s    

2021-08-18 13:23:34 (104 MB/s) - ‘mobilebert_squad_savedmodels.tar.gz’ saved [184730530/184730530]

mobilebert_squad_savedmodels/
mobilebert_squad_savedmodels/quant_saved_model/
mobilebert_squad_savedmodels/quant_saved_model/saved_model.pb
mobilebert_squad_savedmodels/quant_saved_model/variables/
mobilebert_squad_savedmodels/quant_saved_model/variables/variables.index
mobilebert_squad_savedmodels/quant_saved_model/variables/variables.da

In [2]:
!pip install coremltools==4.1

     |████████████████████████████████| 3.4 MB 32.4 MB/s 
  Created wheel for attr: filename=attr-0.3.1-py3-none-any.whl size=2457 sha256=fac18e3bf24556d0c1efaee82621d7fbffc67780af0192e040d4287ebc736a09
  Stored in directory: /root/.cache/pip/wheels/3b/5d/58/41fbe92f47031641008bd8559ee89e58bf0f123f9c18dea1cb
Successfully built attr


In [3]:
%tensorflow_version 2.x

In [4]:
import sys
print(sys.version)

import tensorflow as tf
print(tf.__version__)

import coremltools as ct
print(ct.__version__)

3.7.11 (default, Jul  3 2021, 18:01:19) 
[GCC 7.5.0]
2.6.0


4.1


# Inspect the TF model

In [5]:
def load_saved_model(path):
  print(f"Loading saved_model.pb from {path}")
  the_graph = tf.Graph()
  with tf.compat.v1.Session(graph=the_graph) as sess:
    tags = [tf.compat.v1.saved_model.tag_constants.SERVING]
    tf.compat.v1.saved_model.loader.load(sess, tags, path)
  return the_graph


def export_ops_name(the_graph, filename):
  with open(filename, "w") as text_file:
    ops = the_graph.get_operations()
    N = len(ops)
    for i in range(N):
      text_file.write('\n\nop id {} , op type: "{}"'.format(str(i), ops[i].type))
      
      text_file.write('\ninput(s):'),
      for x in ops[i].inputs:
        text_file.write("name = {}, shape: {}, ".format(x.name, x.get_shape()))
      
      text_file.write('\noutput(s):'),
      for x in ops[i].outputs:
        text_file.write("name = {}, shape: {},".format(x.name, x.get_shape()))
  print('Exported to:', filename)    


saved_model_dir = 'mobilebert_squad_savedmodels/float'

export_ops_name(load_saved_model(saved_model_dir),
                'mobilebert_squad_savedmodels_ops.txt')

Loading saved_model.pb from mobilebert_squad_savedmodels/float
Exported to: mobilebert_squad_savedmodels_ops.txt


In [10]:
from tensorflow.python.tools import strip_unused_lib
from tensorflow.python.framework import dtypes
from tensorflow.python.platform import gfile


def print_node_name(graph_def):
  nodes = [n.name for n in graph_def.node]
  print('nodes', len(nodes), nodes)
  print(nodes[0], '->', nodes[-1])


def optimize_graph(graph_def, 
                   input_node_names, 
                   output_node_names,
                   export_dir):

  print('nodes before strip_unused_lib')
  print_node_name(graph_def)
  
  graph_def = strip_unused_lib.strip_unused(
          input_graph_def = graph_def,
          input_node_names = input_node_names,
          output_node_names = output_node_names,
          placeholder_type_enum = dtypes.float32.as_datatype_enum)
  
  print('nodes after strip_unused_lib')
  print_node_name(graph_def)

  builder = tf.compat.v1.saved_model.builder.SavedModelBuilder(export_dir)
  sigs = {}
  
  with tf.compat.v1.Session(graph=tf.Graph()) as sess:
      tf.import_graph_def(graph_def, name="")
      g = tf.compat.v1.get_default_graph()
      inputs = {n:g.get_tensor_by_name(n+':0') for n in input_node_names}
      outputs = {n:g.get_tensor_by_name(n+':0') for n in output_node_names}

      sigs[signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY] = \
        tf.compat.v1.saved_model.signature_def_utils.predict_signature_def(inputs, outputs)

      builder.add_meta_graph_and_variables(sess, [tag_constants.SERVING], signature_def_map=sigs)
      builder.save()


optimized_saved_model_dir = 'optimized_saved_model'

the_graph = load_saved_model(saved_model_dir)
optimize_graph(the_graph.as_graph_def(), 
               ['input_ids', 'input_mask', 'segment_ids'], 
               ['start_logits', 'end_logits'],
               optimized_saved_model_dir)

Loading saved_model.pb from mobilebert_squad_savedmodels/float
nodes before strip_unused_lib
nodes 10763 ['global_step/Initializer/zeros', 'global_step', 'global_step/IsInitialized/VarIsInitializedOp', 'global_step/Assign', 'global_step/Read/ReadVariableOp', 'input_ids', 'input_mask', 'segment_ids', 'Cast', 'bert/embeddings/ExpandDims/dim', 'bert/embeddings/ExpandDims', 'bert/embeddings/word_embeddings/Initializer/truncated_normal/shape', 'bert/embeddings/word_embeddings/Initializer/truncated_normal/mean', 'bert/embeddings/word_embeddings/Initializer/truncated_normal/stddev', 'bert/embeddings/word_embeddings/Initializer/truncated_normal/TruncatedNormal', 'bert/embeddings/word_embeddings/Initializer/truncated_normal/mul', 'bert/embeddings/word_embeddings/Initializer/truncated_normal', 'bert/embeddings/word_embeddings', 'bert/embeddings/word_embeddings/Assign', 'bert/embeddings/word_embeddings/read', 'bert/embeddings/embedding_lookup/axis', 'bert/embeddings/embedding_lookup', 'bert/embed

ValueError: ignored

# Note
We can use TF1, save the graph_def as frozen_graph without error. But later when convert the frozen_graph to Core ML we will get the same error because coremltools also imports the graph_def first.